In [22]:
from langgraph.graph import StateGraph,START,END
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI 
from langchain_core.messages import BaseMessage, HumanMessage
from dotenv import load_dotenv
from typing import TypedDict , Literal , Annotated
import os
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
load_dotenv()

True

In [2]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash",temperature=0.5)

In [3]:
class chatState(TypedDict):
    message : Annotated[list[BaseMessage], add_messages]

In [5]:
def chat_model(state:chatState):
    messages = state['message']
    response = llm.invoke(messages)
    return {'message': response}

In [23]:
check = MemorySaver()
graph = StateGraph(chatState)

graph.add_node("Chatbot",chat_model)
graph.add_edge(START,"Chatbot")
graph.add_edge("Chatbot",END)

workflow = graph.compile(checkpointer = check)

In [17]:
initial_state = {
    'message' : [HumanMessage(content="Hello, how are you?")]
}
result = workflow.invoke(initial_state)
print(result['message'][-1].content)

Hello! As an AI, I don't have feelings or a physical state, so I'm always functioning perfectly. Thank you for asking!

How are you today?


In [24]:
thread_id = '1'
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit', 'bye']:
        break
    initial_state = {
        'message' : [HumanMessage(content=user_input)]
    }
    config = {'configurable':{'thread_id':thread_id}}
    result = workflow.invoke(initial_state,config = config)
    print("You:", user_input)
    print("Bot:", result['message'][-1].content)


You: hi my name is rohit
Bot: Hello Rohit! It's nice to meet you. I'm a large language model, an AI. How can I help you today?
You: do you know my name
Bot: Yes, I do! You told me your name is **Rohit**.
You: can you add 10 to 45
Bot: Yes, I can!

10 + 45 = **55**
You: multiply the result by 2
Bot: Okay, let's do that.

The previous result was 55.

55 * 2 = **110**
